# Intro to Multi-Agent Systems

> *This notebook should work well with the **`conda_python3`** kernel in SageMaker Studio on ml.t3.medium instance*

---

In this notebook, we will extend the concepts of agents to multi-agent systems. Using multiple agents allows us to create more specialized agents that can work together to solve complex problems. Unlike a single, jack-of-all-trades agent, multi-agent systems can be more efficient and effective at solving problems that require a variety of skills.
In the example below, we'll build a investment and retirement planning system that uses multiple agents to help a user plan for their future. The system will be comprised of three agents:
- Manager Agent: This agent will be responsible for creating the overall plan and delegating tasks to the other agents.
- Investment Analysis Agent: This agent will be responsible for analyzing potential investments
- Retirement Planning Agent: This agent will be responsible for analyzing whether retirement goals can be met given a investment and savings strategy.


---

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

> 🚨 **Caution** You may get an exception running the cell bellow. If that's the case, please restart the kernel by clicking **Kernell** -> **Restart Kernel**. Alternatively click the refresh icon on the notebook toolbar above

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "us.amazon.nova-pro-v1:0",
]
validate_model_access(required_models)

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel
from agent_tools import get_ticker_data, get_fred_data, run_ols_regression, forecast_volatility, simulate_investment_roi, simulate_retirement, get_today

In [ ]:
# MODEL_ID = "bedrock/us.amazon.nova-pro-v1:0"
MODEL_ID = "bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"
model = LiteLLMModel(model_id=MODEL_ID, temperature=0)

investment_tools = [
    get_ticker_data,         # get stock data for a given ticker and date range
    get_fred_data,           # get FRED data for a given series and date range
    get_today,               # get the current date
    run_ols_regression,      # run an OLS regression on dependent and independent variables
    simulate_investment_roi, # simulate the ROI of an investment by running a Monte Carlo simulation on historic returns
    forecast_volatility,     # forecast the volatility of a stock using GARCH
    DuckDuckGoSearchTool(),  # search the web using DuckDuckGo
]
retirement_tools = [
    get_today,               # get the current date
    simulate_investment_roi, # simulate the ROI of an investment by running a Monte Carlo simulation on historic returns
    simulate_retirement,     # simulate the likelihood of running out of money in retirement
]


investment_analyst_agent = CodeAgent(
    tools=investment_tools,
    model=model,
    additional_authorized_imports=["pandas", "json", "numpy"],
    name="investment_analyst_agent",
    description="A research agent that specializes in analyzing stock performance, computing technical indicators, and forecasting volatility.",
)

retirement_planner_agent = CodeAgent(
    tools=retirement_tools,
    model=model,
    additional_authorized_imports=["pandas", "json", "numpy"],
    name="retirement_planner_agent",
    description="An investment analyst agent that can analyze investment plans and help with retirement planning.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[investment_analyst_agent, retirement_planner_agent],
)

In [ ]:
manager_agent.run("Should I invest in bitcoin or a portfolio of FAANG stocks?")

In [ ]:
manager_agent.run("I have $1000000 in savings, and plan to spend roughly $60k per year in retirement. Will this money last me for 30 years?")

In [ ]:
manager_agent.run("I'm 30 years old and have $5K that I'd like to invest into an S&P 500 index fund. How much should I contribute each month so that I can retire at 60 and live comfortably for at least 30 years?")